# Lab 2.1 – RAG with Watsonx, Chroma, and LangChain (Google Colab Version)

![watsonx](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)

---

## 🚀 Run this notebook in Google Colab

**Prerequisites:**
- IBM Cloud API Key ([Create one here](https://cloud.ibm.com/iam/apikeys))
- Watsonx Project ID ([Find it in your watsonx.ai project](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp))

This notebook demonstrates **Retrieval Augmented Generation (RAG)** using:
- **Watsonx.ai** for LLM inference (Granite models)
- **Chroma** as vector database
- **LangChain** for RAG orchestration

---

## Step 1: Install Dependencies

First, we install all required libraries for this RAG implementation.

In [12]:
# Install all required dependencies
!pip install -qU "pydantic<2" # Downgrade pydantic to avoid conflict with spacy
!pip install -qU spacy==2.3.7 # Install compatible spacy version
!pip install -qU wget
!pip install -qU "langchain>=0.3,<0.4"
!pip install -qU "ibm_watsonx_ai>=1.1.22"
!pip install -qU "langchain_ibm>=0.3,<0.4"
!pip install -qU "langchain_chroma>=0.1,<0.2"
!pip install -qU langchain-community

print("✅ All dependencies installed successfully!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.24 which is incompatible.
langchain-core 1.0.5 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.24 which is incompatible.
pydantic-settings 2.12.0 requires pydantic>=2.7.0, but you have pydantic 1.10.24 which is incompatible.
albumentations 2.0.8 requires pydantic>=2.9.2, but you have pydantic 1.10.24 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 1.10.24 which is incompatible.
thinc 8.3.8 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.24 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelem

## Step 2: Configure Credentials

Enter your IBM Cloud API Key and Watsonx Project ID when prompted.

In [2]:
import os
import getpass

# Prompt for Watsonx credentials
watsonx_api_key = getpass.getpass("Enter IBM Cloud API Key: ")
project_id = getpass.getpass("Enter Watsonx Project ID: ")
url = "https://us-south.ml.cloud.ibm.com"  # Default region

# Store in environment variables
os.environ["WATSONX_APIKEY"] = watsonx_api_key
os.environ["PROJECT_ID"] = project_id
os.environ["WATSONX_URL"] = url

print("✅ Credentials configured successfully!")

Enter IBM Cloud API Key: ··········
Enter Watsonx Project ID: ··········
✅ Credentials configured successfully!


## Step 3: Initialize Watsonx API Client

In [3]:
from ibm_watsonx_ai import Credentials, APIClient

credentials = Credentials(
    url=url,
    api_key=watsonx_api_key,
)

api_client = APIClient(credentials=credentials, project_id=project_id)

print("✅ Watsonx API Client initialized!")

✅ Watsonx API Client initialized!


## Step 4: Download Sample Data

We'll use the State of the Union speech as our knowledge base.

In [4]:
import wget

filename = 'state_of_the_union.txt'
url_data = 'https://raw.github.com/IBM/watson-machine-learning-samples/master/cloud/data/foundation_models/state_of_the_union.txt'

if not os.path.isfile(filename):
    wget.download(url_data, out=filename)
    print(f"\n✅ Downloaded {filename}")
else:
    print(f"✅ {filename} already exists")


✅ Downloaded state_of_the_union.txt


## Step 5: Build Knowledge Base with Chroma

We'll:
1. Load the document
2. Split it into chunks
3. Create embeddings using Watsonx
4. Store in Chroma vector database

In [21]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma

# Load document
loader = TextLoader(filename)
documents = loader.load()

# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(f"✅ Split document into {len(texts)} chunks")

✅ Split document into 42 chunks


## Step 6: Create Embeddings with Watsonx

View available embedding models:

In [ ]:
# Show available embedding models
api_client.foundation_models.EmbeddingModels.show()

In [ ]:
from langchain_ibm import WatsonxEmbeddings

embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id
)

# Create Chroma vector store
docsearch = Chroma.from_documents(texts, embeddings)

print("✅ Vector database created with Chroma!")

## Step 7: Initialize Watsonx LLM (Granite Model)

In [ ]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM

model_id = ModelTypes.GRANITE_13B_CHAT_V2

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

watsonx_granite = WatsonxLLM(
    model_id=model_id.value,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

print("✅ Watsonx Granite LLM initialized!")

## Step 8: Create RAG Chain

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=watsonx_granite,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

print("✅ RAG chain created!")

## Step 9: Test the RAG System

In [ ]:
query = "What did the president say about Ketanji Brown Jackson?"
result = qa.invoke(query)

print("\n" + "="*80)
print(f"Question: {result['query']}")
print("="*80)
print(f"Answer: {result['result']}")
print("="*80)

## Step 10: Try Your Own Questions!

In [ ]:
# Try asking different questions about the State of the Union
your_question = input("Enter your question: ")
result = qa.invoke(your_question)

print("\n" + "="*80)
print(f"Question: {result['query']}")
print("="*80)
print(f"Answer: {result['result']}")
print("="*80)

---

## Summary

You successfully completed this notebook! You learned how to:

✅ Set up Watsonx.ai credentials in Google Colab  
✅ Create embeddings using Watsonx embedding models  
✅ Build a vector database with Chroma  
✅ Implement RAG using LangChain and Watsonx Granite models  

For more information:
- [Watsonx.ai Documentation](https://ibm.github.io/watsonx-ai-python-sdk/samples.html)
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)

---

**Copyright © 2023, 2024 IBM. This notebook and its source code are released under the terms of the MIT License.**

# Task
Uninstall `pydantic` and `spacy`, then install `pydantic<2` and `spacy==2.3.7`, and finally re-run the `TextLoader` cell to resolve the `KeyError: '__reduce_cython__'`.

## Uninstall conflicting packages

### Subtask:
Uninstall `pydantic` and `spacy` to ensure no conflicting versions are present.


**Reasoning**:
To address potential dependency conflicts, I will uninstall `pydantic` and `spacy` before reinstalling specific versions.



In [15]:
pip uninstall -y pydantic
pip uninstall -y spacy

print("✅ Uninstalled pydantic and spacy.")

SyntaxError: invalid syntax (ipython-input-517418151.py, line 1)

**Reasoning**:
The previous command resulted in a `SyntaxError` because shell commands like `pip` need to be prefixed with `!` when executed within a Python code cell in a Colab environment. I will correct this by adding the `!` prefix.



In [16]:
!
pip uninstall -y pydantic
!pip uninstall -y spacy

print("✅ Uninstalled pydantic and spacy.")

SyntaxError: invalid syntax (ipython-input-2912738515.py, line 2)

**Reasoning**:
The previous attempt to fix the `SyntaxError` still contained an incorrect application of the `!` prefix. Each shell command, including `pip uninstall`, must be individually prefixed with `!` to execute correctly within a Colab Python cell.



In [17]:
import os

# Uninstall pydantic if it exists
!pip uninstall -y pydantic

# Uninstall spacy if it exists
!pip uninstall -y spacy

print("✅ Uninstalled pydantic and spacy.")

Found existing installation: pydantic 2.12.4
Uninstalling pydantic-2.12.4:
  Successfully uninstalled pydantic-2.12.4
Found existing installation: spacy 3.8.8
Uninstalling spacy-3.8.8:
  Successfully uninstalled spacy-3.8.8
✅ Uninstalled pydantic and spacy.


**Reasoning**:
The conflicting packages (`pydantic` and `spacy`) have been successfully uninstalled. Now, I will proceed to install the specific versions of `pydantic<2` and `spacy==2.3.7` as required by the main task to resolve the dependency issues.



In [18]:
import os

!pip install -qU "pydantic<2" # Downgrade pydantic to avoid conflict with spacy
!pip install -qU spacy==2.3.7 # Install compatible spacy version
!pip install -qU wget
!pip install -qU "langchain>=0.3,<0.4"
!pip install -qU "ibm_watsonx_ai>=1.1.22"
!pip install -qU "langchain_ibm>=0.3,<0.4"
!pip install -qU "langchain_chroma>=0.1,<0.2"
!pip install -qU langchain-community

print("✅ All dependencies installed successfully!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.24 which is incompatible.
langchain-core 1.0.5 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.24 which is incompatible.
pydantic-settings 2.12.0 requires pydantic>=2.7.0, but you have pydantic 1.10.24 which is incompatible.
albumentations 2.0.8 requires pydantic>=2.9.2, but you have pydantic 1.10.24 which is incompatible.
gradio 5.49.1 requires pydantic<2.12,>=2.0, but you have pydantic 1.10.24 which is incompatible.
thinc 8.3.8 requires pydantic<3.0.0,>=2.0.0, but you have pydantic 1.10.24 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelem

**Reasoning**:
The specified versions of `pydantic` and `spacy` have been installed. Now, I will re-run the `TextLoader` cell (cell_id `RcPOxSU35Hzb`) to verify if the `KeyError: '__reduce_cython__'` has been resolved by the dependency changes.



In [19]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma

# Load document
loader = TextLoader(filename)
documents = loader.load()

# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(f"✅ Split document into {len(texts)} chunks")

SyntaxError: invalid syntax (ipython-input-2083599.py, line 3)

**Reasoning**:
The previous code block resulted in a `SyntaxError` because the import statement `from langchain_chroma` was incomplete. I will correct this by specifying `import Chroma` from `langchain_chroma` to properly import the necessary class.



In [20]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma

# Load document
loader = TextLoader(filename)
documents = loader.load()

# Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

print(f"✅ Split document into {len(texts)} chunks")

✅ Split document into 42 chunks


## Continue notebook execution

### Subtask:
Create embeddings using Watsonx and build the Chroma vector database.


## Summary:

### Data Analysis Key Findings

*   The initial attempts to uninstall `pydantic` and `spacy` failed due to missing `!` prefixes for `pip` commands within the Python environment.
*   `pydantic` (version 2.12.4) and `spacy` (version 3.8.8) were successfully uninstalled after correcting the `pip` command syntax.
*   The required versions, `pydantic<2` and `spacy==2.3.7`, along with other dependencies, were installed, despite numerous `pip` dependency conflict warnings during the installation process.
*   Rerunning the `TextLoader` cell initially failed due to a `SyntaxError` caused by an incomplete `import` statement (`from langchain_chroma`).
*   After correcting the import statement to `from langchain_chroma import Chroma`, the `TextLoader` cell successfully executed, loading the document and splitting it into 42 chunks, thereby resolving the original `KeyError: '__reduce_cython__'`.

### Insights or Next Steps

*   **Dependency Management:** Strict version control for libraries like `pydantic` and `spacy` is critical to prevent `KeyError` or similar compatibility issues in complex environments.
*   **Proceed with Vector Database Creation:** With the `TextLoader` now functional, the next step is to continue with creating embeddings using Watsonx and building the Chroma vector database.
